In [1]:
DATE = "2021-05-21"
# TASK = "phage-k-pangenome"
DIR = mkpath("$(homedir())/$(DATE)")

"/Users/cameronprybol/2021-05-21"

In [2]:
pkgs = [
"LightGraphs",
"MetaGraphs",
"BioSequences",
"uCSV",
"DataFrames",
"FASTX",
"HTTP",
"CodecZlib",
"DataStructures",
"Revise",
"ProgressMeter",
"BenchmarkTools"
]

import Pkg
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $(basename(pkg))"))
end

import Mycelia

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
No Changes to `~/.julia/dev/Mycelia/Project.toml`
No Changes to `~/.julia/dev/Mycelia/Manifest.toml`
┌ Info: Precompiling Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd]
└ @ Base loading.jl:1278


In [3]:
ncbi_staph_phage_metadata = DataFrames.DataFrame(uCSV.read("$(dirname(pwd()))/metadata/ncbi-staph-phage.csv", header=1, quotes='"', typedetectrows=100)...)
ncbi_staph_phage_metadata = ncbi_staph_phage_metadata[ncbi_staph_phage_metadata[!, "Nuc_Completeness"] .== "complete", :]
# ncbi_staph_phage_metadata = ncbi_staph_phage_metadata[ncbi_staph_phage_metadata[!, "Sequence_Type"] .== "RefSeq", :]
ncbi_staph_phage_metadata = ncbi_staph_phage_metadata[ncbi_staph_phage_metadata[!, "Genus"] .== "Kayvirus", :];
show(ncbi_staph_phage_metadata[1:3, :], allcols=true)

3×21 DataFrame
│ Row │ Accession │ SRA_Accession │
│     │ String    │ String        │
├─────┼───────────┼───────────────┤
│ 1   │ NC_047720 │               │
│ 2   │ NC_047721 │               │
│ 3   │ NC_047722 │               │

│ Row │ Submitters                                                                                                                                                                        │
│     │ String                                                                                                                                                                            │
├─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 1   │ Vandersteegen,K., Mattheus,W., Ceyssens,P.J., Bilocq,F., De Vos,D., Pirnay,J.P., Noben,J.P., Merabishvili,M., Lipinska,U., Hermans,K., Lavigne,R.                                 │
│ 2   │ Kirby,A.

In [4]:
size(ncbi_staph_phage_metadata)

(64, 21)

In [5]:
# # 11, 17 is too small
# for k in [13, 17, 23, 31, 41, 53]
#     @show k
#     outgraph = "$(DIR)/simplified-kayvirus.k_$(k).all_ncbi.gfa"
#     # if !isfile(outgraph)
#     accessions = ncbi_staph_phage_metadata[!, "Accession"]
#     fastx_iterator = (Mycelia.get_sequence(db="nuccore", accession=accession) for accession in accessions)
#     KMER_TYPE = BioSequences.BigDNAMer{k}
#     @time kmer_graph = Mycelia.fastx_to_kmer_graph(KMER_TYPE, fastx_iterator)

#     @info "simplifying kmer graph"
#     @info "resolving untigs..."
#     @time untigs = Mycelia.resolve_untigs(kmer_graph)

#     @info "determining untig orientations..."
#     @time oriented_untigs = Mycelia.determine_oriented_untigs(kmer_graph, untigs)

#     simplified_graph = MetaGraphs.MetaDiGraph(length(oriented_untigs))
#     MetaGraphs.set_prop!(simplified_graph, :k, kmer_graph.gprops[:k])
#     @info "initializing graph node metadata"
#     ProgressMeter.@showprogress for (vertex, untig) in enumerate(oriented_untigs)
#         MetaGraphs.set_prop!(simplified_graph, vertex, :sequence, untig.sequence)
#         MetaGraphs.set_prop!(simplified_graph, vertex, :path, untig.path)
#         MetaGraphs.set_prop!(simplified_graph, vertex, :orientations, untig.orientations)
#         MetaGraphs.set_prop!(simplified_graph, vertex, :evidence, untig.evidence)
#     end

#     # determine oriented edges of simplified graph
#     simplified_untigs = Vector{Pair{Pair{Int64,Bool},Pair{Int64,Bool}}}(undef, length(LightGraphs.vertices(simplified_graph)))
#     @info "creating simplified unitgs to help resolve connections"
#     # use a pre-allocated array here to speed up
#     ProgressMeter.@showprogress for vertex in LightGraphs.vertices(simplified_graph)
#         in_kmer = simplified_graph.vprops[vertex][:path][1] => simplified_graph.vprops[vertex][:orientations][1]
#         out_kmer = simplified_graph.vprops[vertex][:path][end] => simplified_graph.vprops[vertex][:orientations][end]
#     #     @show vertex, in_kmer, out_kmer
#         simplified_untigs[vertex] = in_kmer => out_kmer
#     #     push!(simplified_untigs, )
#     end

#     # make a dictionary mapping endcap to oriented_untig index

#     end_mer_map = Dict()
#     ProgressMeter.@showprogress for (i, oriented_untig) in enumerate(oriented_untigs)
#         end_mer_map[first(oriented_untig.path)] = i
#         end_mer_map[last(oriented_untig.path)] = i
#     end

#     ProgressMeter.@showprogress for (untig_index, oriented_untig) in enumerate(oriented_untigs)
#     #     @show untig_index
#         true_in_overlap = oriented_untig.sequence[1:simplified_graph.gprops[:k]-1]

#         non_backtracking_neighbors = LightGraphs.neighbors(kmer_graph, oriented_untig.path[1])
#         if length(oriented_untig.path) > 1
#             non_backtracking_neighbors = setdiff(non_backtracking_neighbors, oriented_untig.path[2])
#         end
#         for non_backtracking_neighbor in non_backtracking_neighbors
#             neighboring_untig_index = end_mer_map[non_backtracking_neighbor]
#             neighboring_untig = oriented_untigs[neighboring_untig_index]

#             neighbor_true_out_overlap = neighboring_untig.sequence[end-simplified_graph.gprops[:k]+2:end]
#             if neighbor_true_out_overlap == true_in_overlap
#                 e = LightGraphs.Edge(neighboring_untig_index, untig_index)
#     #             o = true => true
#                 o = (source_orientation = true, destination_orientation = true)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end

#             neighbor_false_out_overlap = BioSequences.reverse_complement(neighboring_untig.sequence)[end-simplified_graph.gprops[:k]+2:end]
#             if neighbor_false_out_overlap == true_in_overlap        
#                 e = LightGraphs.Edge(neighboring_untig_index, untig_index)
#     #             o = false => true
#                 o = (source_orientation = false, destination_orientation = true)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end
#         end

#         true_out_overlap = oriented_untig.sequence[end-simplified_graph.gprops[:k]+2:end]
#         non_backtracking_neighbors = LightGraphs.neighbors(kmer_graph, oriented_untig.path[end])
#         if length(oriented_untig.path) > 1
#             non_backtracking_neighbors = setdiff(non_backtracking_neighbors, oriented_untig.path[end-1])
#         end
#         for non_backtracking_neighbor in non_backtracking_neighbors
#             neighboring_untig_index = end_mer_map[non_backtracking_neighbor]
#             neighboring_untig = oriented_untigs[neighboring_untig_index]

#             neighbor_true_in_overlap = neighboring_untig.sequence[1:simplified_graph.gprops[:k]-1]
#             if true_out_overlap == neighbor_true_in_overlap
#                 e = LightGraphs.Edge(untig_index, neighboring_untig_index)
#     #             o = true => true
#                 o = (source_orientation = true, destination_orientation = true)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end

#             neighbor_false_in_overlap = BioSequences.reverse_complement(neighboring_untig.sequence)[1:simplified_graph.gprops[:k]-1]
#             if true_out_overlap == neighbor_false_in_overlap
#                 e = LightGraphs.Edge(untig_index, neighboring_untig_index)
#     #             o = true => false
#                 o = (source_orientation = true, destination_orientation = false)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end
#         end

#         false_in_overlap = BioSequences.reverse_complement(oriented_untig.sequence)[1:simplified_graph.gprops[:k]-1]

#         non_backtracking_neighbors = LightGraphs.neighbors(kmer_graph, oriented_untig.path[end])
#         if length(oriented_untig.path) > 1
#             non_backtracking_neighbors = setdiff(non_backtracking_neighbors, oriented_untig.path[end-1])
#         end
#         for non_backtracking_neighbor in non_backtracking_neighbors
#             neighboring_untig_index = end_mer_map[non_backtracking_neighbor]
#             neighboring_untig = oriented_untigs[neighboring_untig_index]

#             neighbor_true_out_overlap = neighboring_untig.sequence[end-simplified_graph.gprops[:k]+2:end]
#             if neighbor_true_out_overlap == false_in_overlap
#                 e = LightGraphs.Edge(neighboring_untig_index, untig_index)
#     #             o = true => false
#                 o = (source_orientation = true, destination_orientation = false)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end

#             neighbor_false_out_overlap = BioSequences.reverse_complement(neighboring_untig.sequence)[end-simplified_graph.gprops[:k]+2:end]
#             if neighbor_false_out_overlap == false_in_overlap        
#                 e = LightGraphs.Edge(neighboring_untig_index, untig_index)
#     #             o = false => false
#                 o = (source_orientation = false, destination_orientation = false)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end
#         end

#         false_out_overlap = BioSequences.reverse_complement(oriented_untig.sequence)[end-simplified_graph.gprops[:k]+2:end]

#         non_backtracking_neighbors = LightGraphs.neighbors(kmer_graph, oriented_untig.path[1])
#         if length(oriented_untig.path) > 1
#             non_backtracking_neighbors = setdiff(non_backtracking_neighbors, oriented_untig.path[2])
#         end

#         for non_backtracking_neighbor in non_backtracking_neighbors
#             neighboring_untig_index = end_mer_map[non_backtracking_neighbor]
#             neighboring_untig = oriented_untigs[neighboring_untig_index]

#             neighbor_true_in_overlap = neighboring_untig.sequence[1:simplified_graph.gprops[:k]-1]
#             if false_out_overlap == neighbor_true_in_overlap
#                 e = LightGraphs.Edge(untig_index, neighboring_untig_index)
#     #             o = false => true
#                 o = (source_orientation = false, destination_orientation = true)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end

#             neighbor_false_in_overlap = BioSequences.reverse_complement(neighboring_untig.sequence)[1:simplified_graph.gprops[:k]-1]
#             if false_out_overlap == neighbor_false_in_overlap
#                 e = LightGraphs.Edge(untig_index, neighboring_untig_index)
#     #             o = false => false
#                 o = (source_orientation = false, destination_orientation = false)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end
#         end
#     end

#     # ^ > 1 day down to 34 seconds!
#     Mycelia.graph_to_gfa(simplified_graph, outgraph)
# end

In [6]:
for k in [11, 13, 17, 23, 31, 41, 53]
    @show k
    outgraph = "$(DIR)/simplified-kayvirus.k_$(k).all_ncbi.gfa"
    if !isfile(outgraph)
        accessions = ncbi_staph_phage_metadata[!, "Accession"]
        fastx_iterator = (Mycelia.get_sequence(db="nuccore", accession=accession) for accession in accessions)
        KMER_TYPE = BioSequences.BigDNAMer{k}
        @time kmer_graph = Mycelia.fastx_to_kmer_graph(KMER_TYPE, fastx_iterator)
        @time simplified_graph = Mycelia.simplify_kmer_graph(kmer_graph)
#         ^ > 1 day down to 34 seconds!
        Mycelia.graph_to_gfa(simplified_graph, outgraph)
    end
end

┌ Info: assessing kmers
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1850
Progress: 100%|█████████████████████████████████████████| Time: 0:01:05
┌ Info: creating graph
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1864
Progress: 100%|█████████████████████████████████████████| Time: 0:06:12


446.011388 seconds (399.32 M allocations: 49.386 GiB, 49.98% gc time)


┌ Info: simplifying kmer graph
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2174
┌ Info: resolving untigs...
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2175


 20.322725 seconds (41.61 M allocations: 2.693 GiB, 68.38% gc time)


┌ Info: determining untig orientations...
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2178


...

 83.782801 seconds (45.64 M allocations: 2.962 GiB, 87.24% gc time)


┌ Info: determining untig orientations...
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2178


  6.205942 seconds (12.35 M allocations: 767.760 MiB, 9.28% gc time)


┌ Info: initializing graph node metadata
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2183
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
┌ Info: creating simplified unitgs to help resolve connections
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2193
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
Progress: 100%|█████████████████████████████████████████| Time: 0:00:19


110.306306 seconds (64.19 M allocations: 4.185 GiB, 84.23% gc time)


In [ ]:
# 13 is too small 96k nodes 134k edges 0.71:1
# 17 is too small nodes 48k edges 64k 0.75:1
# 23 is useable! 40k nodes 54k edges 0.74:1
# 31 is useable! 33k nodes 45k edges 0.73:1
# 41 is useable! 27k nodes 37k edges 0.72:1
# 53 is usable! 22k nodes 30k edges 0.73:1

In [ ]:
# 18.262774 seconds (43.09 M allocations: 3.537 GiB, 30.36% gc time)
# for first three genomes using giant function

# 14.711297 seconds (25.01 M allocations: 3.098 GiB, 31.72% gc time)
# for first three genomes using external function call to inner code

# # 14.479504 seconds (25.01 M allocations: 3.099 GiB, 27.59% gc time)
# for first three genomes using inlined external function

In [ ]:
# 8
# 17
# 26

In [ ]:
# k = 13

# ┌ Info: assessing kmers
# └ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1761
# Progress: 100%|█████████████████████████████████████████| Time: 0:02:02
# ┌ Info: creating graph
# └ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1775
# Progress: 100%|█████████████████████████████████████████| Time: 1:36:28

# 6009.693184 seconds (791.81 M allocations: 57.006 GiB, 93.91% gc time)
#  69.522944 seconds (24.97 M allocations: 1.609 GiB, 91.99% gc time)
#   2.609659 seconds (6.36 M allocations: 334.051 MiB, 15.21% gc time)

In [ ]:
# k = 17

# ┌ Info: assessing kmers
# └ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1761
# Progress: 100%|█████████████████████████████████████████| Time: 0:02:08
# ┌ Info: creating graph
# └ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1775
# Progress: 100%|█████████████████████████████████████████| Time: 2:35:09

# 9662.350843 seconds (798.58 M allocations: 58.014 GiB, 96.20% gc time)
# 100.723128 seconds (28.30 M allocations: 1.830 GiB, 93.68% gc time)
#   3.381248 seconds (7.51 M allocations: 404.454 MiB, 18.11% gc time)

In [ ]:
# k = 23

# ┌ Info: assessing kmers
# └ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1761
# Progress: 100%|█████████████████████████████████████████| Time: 0:01:02
# ┌ Info: creating graph
# └ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1775
# Progress: 100%|█████████████████████████████████████████| Time: 3:07:10

# 11609.072807 seconds (805.61 M allocations: 58.927 GiB, 95.85% gc time)
#  71.810824 seconds (32.20 M allocations: 2.085 GiB, 89.64% gc time)
#   4.272813 seconds (8.61 M allocations: 470.110 MiB, 18.78% gc time)

In [ ]:
# k = 31

# ┌ Info: assessing kmers
# └ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1761
# Progress: 100%|█████████████████████████████████████████| Time: 0:02:24
# ┌ Info: creating graph
# └ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1775
# Progress: 100%|█████████████████████████████████████████| Time: 13:28:45

# 34069.539714 seconds (813.42 M allocations: 60.062 GiB, 97.42% gc time)